# Poppy Universe – Layer 3: Master Matrix Integration

Welcome to the **Poppy Universe Layer 3 – Master notebook**!  
This notebook integrates the **star, planet, and moon matrix predictions** into a single unified dataset for the recommendation engine. It ensures all user × type scores are combined for Layer 3 semantic ranking.

> Note: This notebook currently uses **simulated user interactions** to merge the matrices.  
> Once we have enough real interactions, the same pipeline will process actual user data for production recommendations.

---

## Goals

1. **Load Layer 3 predictions from all notebooks**  
   - Stars, Planets, and Moons  

2. **Merge predicted matrices into a single user × type DataFrame**  
   - Users in rows, all category values as columns  
   - Fill missing values with 0 or appropriate defaults  

3. **Prepare final CSV for the engine**  
   - Clean schema, sorted columns  
   - Save to Output_Data for consumption by C# engine  

4. **Optional analysis & visualization**  
   - Heatmaps of top types per user  
   - Summary statistics across categories

---

## Folder & File References

- **../../Output_Data/Layer3_Star_Predictions.csv** → Star matrix predictions  
- **../../Output_Data/Layer3_Planet_Predictions.csv** → Planet matrix predictions  
- **../../Output_Data/Layer3_Moon_Predictions.csv** → Moon matrix predictions  
- **../../Output_Data/Layer3_Master_Predictions.csv** → Final merged predictions for engine  
- **Plots/** → Optional heatmaps or visualizations

---

> Note: This notebook focuses **on merging all Layer 3 matrices**. Individual notebooks for Stars, Planets, and Moons remain separate to allow independent updates before final integration.


## 0) Imports

In [1]:
import pandas as pd
import numpy as np
import papermill as pm

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

In [2]:
# --- CONFIG / INPUT ---
# Simulated CSV paths (fallback)
sim_star_path   = '../Files/Layer3_Star_Predictions.csv'
sim_planet_path = '../Files/Layer3_Planet_Predictions.csv'
sim_moon_path   = '../Files/Layer3_Moon_Predictions.csv'

# Backend-provided full interactions table (can be None)
backend_interactions_df = None  # backend sends 1 table with all interactions

# --- MINIMUM REQUIREMENTS ---
MIN_UNIQUE_USERS = 20
MIN_INTERACTIONS_PER_TYPE = 10  # per category type

# --- HELPER FUNCTION TO RUN NOTEBOOKS ---
def run_notebook_with_data(nb_path, interactions_df=None, sim_csv=None, param_name='backend_df'):
    """
    Runs a notebook via papermill, passing the interactions DataFrame as a parameter.
    Only runs if interactions_df meets minimum requirements.
    """
    if interactions_df is not None:
        unique_users = interactions_df['User_ID'].nunique()
        interactions_per_type = interactions_df['Category_Type'].value_counts().min()
        if unique_users >= MIN_UNIQUE_USERS and interactions_per_type >= MIN_INTERACTIONS_PER_TYPE:
            print(f"Running {nb_path} with backend data...")
            pm.execute_notebook(
                nb_path,
                nb_path,  # overwrite notebook
                parameters={param_name: interactions_df}
            )
        else:
            print(f"Skipping {nb_path}; backend data too sparse (users: {unique_users}, min interactions/type: {interactions_per_type})")
    else:
        print(f"Skipping {nb_path}; backend data missing. Using existing CSV ({sim_csv})")

# --- RUN THE COMPONENT NOTEBOOKS (ONLY IF BACKEND DATA IS VALID) ---
run_notebook_with_data('Star_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_star_path)
run_notebook_with_data('Planet_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_planet_path)
run_notebook_with_data('Moon_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_moon_path)

# --- LOAD THE FINAL PREDICTIONS ---
star_df   = pd.read_csv(sim_star_path)
planet_df = pd.read_csv(sim_planet_path)
moon_df   = pd.read_csv(sim_moon_path)

Skipping Star_Matrix_Model.ipynb; backend data missing. Using existing CSV (../Files/Layer3_Star_Predictions.csv)
Skipping Planet_Matrix_Model.ipynb; backend data missing. Using existing CSV (../Files/Layer3_Planet_Predictions.csv)
Skipping Moon_Matrix_Model.ipynb; backend data missing. Using existing CSV (../Files/Layer3_Moon_Predictions.csv)


## 2) check Star, Planet, and Moon Predictions

In [3]:
# Quick check
print("Stars:", star_df.shape)
print("Planets:", planet_df.shape)
print("Moons:", moon_df.shape)

Stars: (100, 8)
Planets: (100, 5)
Moons: (100, 11)


In [4]:
star_df.head()

,User_ID,A,B,F,G,K,M,O
0,1,5.261151,4.628608,3.639428,3.149804,4.789282,4.882130,3.897102
1,2,3.834061,5.071705,4.671911,4.235918,4.116431,5.227182,4.415666
2,3,3.190381,4.374800,4.894918,4.908559,3.677740,4.098439,4.270600
3,4,4.489275,5.047683,4.541268,4.132878,4.522362,5.181424,4.415533
4,5,5.105428,3.062459,4.024846,4.637619,4.640830,2.260102,3.661674


In [5]:
planet_df.head()

,User_ID,Dwarf Planet,Gas Giant,Ice Giant,Terrestrial
0,1,4.710570,4.136557,5.032102,5.042292
1,2,4.860355,4.620260,4.830528,4.785199
2,3,5.107094,4.934059,4.965450,4.959903
3,4,4.859762,4.616647,4.832396,4.788828
4,5,4.926277,4.989948,4.015977,4.988191


In [6]:
moon_df.head()

,User_ID,Earth,Eris,Haumea,Jupiter,Makemake,Mars,Neptune,Pluto,Saturn,Uranus
0,1,0.848477,4.106953,4.645544,6.324609,2.800636,4.683168,3.498272,4.044053,4.970344,3.304885
1,2,4.869174,3.983841,3.926422,4.081223,4.220678,2.393614,5.033481,3.926184,3.462591,3.868368
2,3,4.989743,3.994149,3.676311,2.653212,4.323451,4.767981,3.040614,5.028482,3.692991,3.738117
3,4,4.979077,4.860061,4.770785,4.579445,4.824919,4.985730,4.436822,5.489120,4.712128,4.460553
4,5,4.068326,4.687756,4.765528,5.121356,4.368546,4.617763,4.473047,5.032226,4.729884,4.220028


## 3) Merge Star, Planet, and Moon matrices

In [7]:
# Drop User_ID from planet and moon so it doesn't duplicate
planet_values = planet_df.drop('User_ID', axis=1)
moon_values = moon_df.drop('User_ID', axis=1)

# Concatenate horizontally
merged_df = pd.concat([star_df, planet_values, moon_values], axis=1)

# Optional: reorder columns so User_ID is first
cols = ['User_ID'] + [c for c in merged_df.columns if c != 'User_ID']
merged_df = merged_df[cols]

# Quick check
print("Merged shape:", merged_df.shape)
merged_df.head()

Merged shape: (100, 22)


,User_ID,A,B,F,G,K,M,O,Dwarf Planet,Gas Giant,...,Earth,Eris,Haumea,Jupiter,Makemake,Mars,Neptune,Pluto,Saturn,Uranus
0,1,5.261151,4.628608,3.639428,3.149804,4.789282,4.882130,3.897102,4.710570,4.136557,...,0.848477,4.106953,4.645544,6.324609,2.800636,4.683168,3.498272,4.044053,4.970344,3.304885
1,2,3.834061,5.071705,4.671911,4.235918,4.116431,5.227182,4.415666,4.860355,4.620260,...,4.869174,3.983841,3.926422,4.081223,4.220678,2.393614,5.033481,3.926184,3.462591,3.868368
2,3,3.190381,4.374800,4.894918,4.908559,3.677740,4.098439,4.270600,5.107094,4.934059,...,4.989743,3.994149,3.676311,2.653212,4.323451,4.767981,3.040614,5.028482,3.692991,3.738117
3,4,4.489275,5.047683,4.541268,4.132878,4.522362,5.181424,4.415533,4.859762,4.616647,...,4.979077,4.860061,4.770785,4.579445,4.824919,4.985730,4.436822,5.489120,4.712128,4.460553
4,5,5.105428,3.062459,4.024846,4.637619,4.640830,2.260102,3.661674,4.926277,4.989948,...,4.068326,4.687756,4.765528,5.121356,4.368546,4.617763,4.473047,5.032226,4.729884,4.220028


## 4) Save final merged matrix for engine

In [8]:
# -----------------------------
# Save final merged matrix for engine
# -----------------------------
merged_df.to_csv('../../../Output_Data/Layer_3_Final_Predictions.csv', index=False)
print("Saved Layer3_Final_Predictions.csv")


Saved Layer3_Final_Predictions.csv
